In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier

# Importing Data

In [2]:
data = pickle.load(open('face_age_classifier.pickle', mode='rb'))

In [3]:
X = np.array(data['data'])
y = np.array(data['label'])

In [4]:
X.shape, y.shape

((9772, 1, 128), (9772,))

In [5]:
X = X.reshape(-1,128)
X.shape

(9772, 128)

# Splitting Data Into Training and Testing Set

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((7817, 128), (1955, 128), (7817,), (1955,))

In [8]:
print(X_test)

[[ 0.04690419 -0.15842931 -0.00356822 ...  0.08689225  0.10842579
   0.08698601]
 [ 0.02629636  0.08009557 -0.11799402 ...  0.00197598  0.05995223
   0.03655804]
 [ 0.03374301  0.08039035  0.09571075 ... -0.01526301  0.23807926
   0.01276315]
 ...
 [-0.06129373  0.02341458 -0.07623527 ...  0.07623772  0.01142412
  -0.01826049]
 [ 0.07416931  0.1847719  -0.05156583 ...  0.08970997  0.14232375
  -0.07423792]
 [ 0.04048434  0.15701409  0.03963395 ... -0.05320634  0.12111621
   0.05769038]]


In [9]:
print(y_test)

['18-25' '36-50' '3-5' ... '51-65' '81-100' '0-2']


# Training Model

## Logistic Regression

In [10]:
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [11]:
def get_report(model, X_train, y_train, X_test, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)

    # F1 Score
    f1_score_train = f1_score(y_train, y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')

    print('Accuracy Train =', accuracy_train)
    print('Accuracy Test =', accuracy_test)
    print('F1 Score Train =', f1_score_train)
    print('F1 Score Test =', f1_score_test)

In [12]:
get_report(logistic_classifier, X_train, y_train, X_test, y_test)

Accuracy Train = 0.5219393629269541
Accuracy Test = 0.4966751918158568
F1 Score Train = 0.46388622885804065
F1 Score Test = 0.4419318036702987


## Support Vector Machine

In [13]:
model_svc = SVC(probability=True)
model_svc.fit(X_train, y_train)

SVC(probability=True)

In [14]:
get_report(model_svc, X_train, y_train, X_test, y_test)

Accuracy Train = 0.6191633619035436
Accuracy Test = 0.507928388746803
F1 Score Train = 0.578705044634469
F1 Score Test = 0.46572904277015487


## Random Forest

In [15]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

RandomForestClassifier()

In [16]:
get_report(model_rf, X_train, y_train, X_test, y_test)

Accuracy Train = 0.9973135473966995
Accuracy Test = 0.4823529411764706
F1 Score Train = 0.9972365056077349
F1 Score Test = 0.4292651905621662


# Voting Classifier

In [17]:
model_voting = VotingClassifier(estimators = [
    ('logisitc', LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
],voting='soft',weights=[2,3,1])

In [18]:
model_voting.fit(X_train,y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('logisitc', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [19]:
get_report(model_voting, X_train, y_train, X_test, y_test)

Accuracy Train = 0.721760266086734
Accuracy Test = 0.5104859335038363
F1 Score Train = 0.6955572227754557
F1 Score Test = 0.4650499691934016


# Parameter Tuning

In [20]:
model_grid = GridSearchCV(model_voting,param_grid={
    'svm__C':[3,5,7,10],
    'svm__gamma':[0.1,0.3,0.5],
    'rf__n_estimators':[5,10,20],
    "rf__max_depth":[3,5,7],
    'voting':['soft','hard']
},scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [21]:
model_grid.fit(X_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  10.4s


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  10.7s


...

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=hard; total time=  11.2s


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=hard; total time=  10.8s


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logisitc',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [22]:
model_best_estimator = model_grid.best_estimator_

In [23]:
model_grid.best_score_

0.5230893420321922

# Saving Model

In [24]:
pickle.dump(model_best_estimator,open('./models/face_age.pkl',mode='wb'))